## **IMPORTING NECESSARY LIBRARIES**





In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf

## IMPORT THE DATASET

In [2]:
data = pd.read_csv('/content/email spam.csv',encoding = 'ISO-8859-1')

In [3]:
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [4]:
data =data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)

In [5]:
data

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


## Preprocessing

In [6]:
data['v1'].replace(['spam', 'ham'],
                        [0, 1], inplace=True)

In [7]:
data

,v1,v2
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,0,This is the 2nd time we have tried 2 contact u...
5568,1,Will Ì_ b going to esplanade fr home?
5569,1,"Pity, * was in mood for that. So...any other s..."
5570,1,The guy did some bitching but I acted like i'd...


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   int64 
 1   v2      5572 non-null   object
dtypes: int64(1), object(1)
memory usage: 87.2+ KB


In [9]:
# converted the sentences into the numpy array of frequency of the tokens
# padding of sequences to get the same length of list
def get_sequences(texts, tokenizer, train=True, max_seq_length=None):
    sequences = tokenizer.texts_to_sequences(texts)
    
    if train == True:
        max_seq_length = np.max(list(map(lambda x: len(x), sequences)))
    
    sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_seq_length, padding='post')
    
    return sequences

In [10]:

def preprocess_inputs(df):
    df = df.copy()
    
   
    
    
    y = df['v1']
    X = df['v2']
    
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    # Create tokenizer
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=30000)
    
    # Fit the tokenizer
    tokenizer.fit_on_texts(X_train)
    
    # Convert texts to sequences
    X_train = get_sequences(X_train, tokenizer, train=True)
    X_test = get_sequences(X_test, tokenizer, train=False, max_seq_length=X_train.shape[1])
    
    return X_train, X_test, y_train, y_test

In [11]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

In [12]:
X_train

array([[3563,  536,   43, ...,    0,    0,    0],
       [1014,    1,  537, ...,    0,    0,    0],
       [ 169,    3,   15, ...,    0,    0,    0],
       ...,
       [ 636,    1,  202, ...,    0,    0,    0],
       [   1,   15, 3536, ...,    0,    0,    0],
       [  89,   89,  261, ...,    0,    0,    0]], dtype=int32)

In [13]:
y_train.value_counts()

1    3371
0     529
Name: v1, dtype: int64

In [14]:
X_train.shape

(3900, 189)

## **MODEL BUILDING**

In [15]:
# building the neural network for the embeddings
inputs = tf.keras.Input(shape=(189,))

embedding = tf.keras.layers.Embedding(
    input_dim=30000,
    output_dim=64
)(inputs)

flatten = tf.keras.layers.Flatten()(embedding)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(flatten)


model = tf.keras.Model(inputs=inputs, outputs=outputs)



model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)


print(model.summary())


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 189)]             0         
                                                                 
 embedding (Embedding)       (None, 189, 64)           1920000   
                                                                 
 flatten (Flatten)           (None, 12096)             0         
                                                                 
 dense (Dense)               (None, 1)                 12097     
                                                                 
Total params: 1,932,097
Trainable params: 1,932,097
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/50
98/98 [==============================] - 8s 65ms/step - loss: 0.3202 - accuracy: 0.8708 - auc: 0.8215 - val_loss: 0.1454 - val_accuracy: 0.9718 - val_auc: 0.9822
Epoch 2/50
98/98 [==============================] - 5s 47ms/step - loss: 0.0925 - accuracy: 0.9763 - auc: 0.9859 - val_loss: 0.0587 - val_accuracy: 0.9859 - val_auc: 0.9967
Epoch 3/50
98/98 [==============================] - 6s 58ms/step - loss: 0.0441 - accuracy: 0.9872 - auc: 0.9977 - val_loss: 0.0440 - val_accuracy: 0.9885 - val_auc: 0.9981
Epoch 4/50
98/98 [==============================] - 6s 62ms/step - loss: 0.0250 - accuracy: 0.9933 - auc: 0.9996 - val_loss: 0.0365 - val_accuracy: 0.9885 - val_auc: 0.9984
Epoch 5/50
98/98 [==============================] - 5s 51ms/step - loss: 0.0146 - accuracy: 0.9965 - auc: 0.9999 - val_loss: 0.0340 - val_accuracy: 0.9872 - val_auc: 0.9984
Epoch 6/50
98/98 [==============================] - 5s 52ms/step - loss: 0.0091 - accuracy: 0.9987 - auc: 1.0000 - val_loss: 0.0344 - v

## **ACCURACY SCORE**

In [17]:
results = model.evaluate(X_test, y_test, verbose=0)

print("    Test Loss: {:.4f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))
print("     Test AUC: {:.4f}".format(results[2]))

    Test Loss: 0.0604
Test Accuracy: 98.39%
     Test AUC: 0.9860


In [18]:
model.predict(X_test).astype(np.int)

53/53 [==============================] - 0s 2ms/step


<ipython-input-18-c86cec8a432b>:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  model.predict(X_test).astype(np.int)


array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [19]:
(np.squeeze(model.predict(X_test).astype(np.int)))

53/53 [==============================] - 0s 2ms/step


<ipython-input-19-3fca8c4cf337>:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (np.squeeze(model.predict(X_test).astype(np.int)))


array([0, 0, 0, ..., 0, 0, 0])

In [20]:
(np.squeeze(np.array(model.predict(X_test) >=0.5, dtype=np.int)) != y_test).sum()

53/53 [==============================] - 0s 2ms/step


<ipython-input-20-21cdbe5dc0d9>:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  (np.squeeze(np.array(model.predict(X_test) >=0.5, dtype=np.int)) != y_test).sum()


27

In [21]:
len(y_test)

1672